# Assessing Subglacial Lakes Dynamics in the Thwaites area, Western Antarctica with CryoSat-2 radar altimetry

CryoSat-2 is a European Space Agency (ESA) satellite launched in 2010. It's main job is to measure changes in the thickness and volume of ice - over land and sea - to help us understand the cryosphere's role in climate change.

In this tutorial, we will use the **CryoTEMPO Land Ice** and **CryoTEMPO-EOLIS** gridded products to study the evolution of the surface elevation of four subglacial lakes underneath Thwaites glacier, in Western Antarctica.
These are altimetry datasets derived from CryoSat-2 measurements using two distinct processing techniques: **Point of Closest Approach** (POCA), and **Swath**.

Along the way you will find some questions. Please provide the answers using the questionnaire at [this link](https://forms.gle/aYiLViyEvqmJfe1N7).

### Environment Setup
To run this notebook, you will need to make sure that the following packages are installed in your python environment (all can be installed via pip/conda):
- matplotlib
- geopandas
- contextily
- ipykernel
- shapely
- specklia

If you are using the Google Colab environment, these packages will be installed in the next cell. Please note this step may take a few minutes.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    %pip install rasterio --no-binary rasterio
    %pip install specklia
    %pip install matplotlib
    %pip install geopandas
    %pip install contextily
    %pip install shapely
    %pip install python-dotenv

In [ ]:
import os
from time import perf_counter
from datetime import datetime, timedelta
import geopandas as gpd
import math
import pprint
import pandas as pd
from shapely import box, Point, Polygon

from typing import List, Tuple
import numpy as np
from numpy.typing import NDArray
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import contextily as ctx
from specklia import Specklia

# Radar altimetry point data

Satellite altimetry measurements are often provided in point data format. The satellite measures the surface elevation of the ground along its orbit by sending an electromagnetic signal and receiving it back, whether it is laser (e.g. ICESat-2) or radar (e.g. CryoSat-2).

Let's get some point data measured by CryoSat-2. The dataset we are going to use now is called **CryoTEMPO Land Ice Thematic Product**. This dataset can be queried via [Specklia](https://specklia.earthwave.co.uk/), a database that provides access to various altimetry datasets.

Specklia is free to use, but an API key needs to be generated and passed to the next cell in order to use it.

In [ ]:
user_api_key = input('Please generate your own key using https://specklia.earthwave.co.uk/ApiKeys and paste it here: ')
client = Specklia(user_api_key)

## Define an area of interest and the time interval for the initial query

To get started, we'll be looking at all the measurements taken during one month (January 2013) within a rectangular area of size 3° x 2° centred on longitude 107°W and latitude 76.5°S (in Western Antarctica).

📌 Note: These coordinates use a global reference system called **WGS84**, identified by the code **EPSG:4326**.

Later in this notebook, we’ll switch to a coordinate system made especially for Antarctica, called **Antarctic Polar Stereographic (EPSG:3031)**, which makes maps and calculations in the polar region easier.

In [ ]:
query_polygon_4326 = Polygon((
    (-108.5, -77.5), (-105.5, -77.5), (-105.5, -75.5), (-108.5, -75.5), (-108.5, -77.5)))
query_extent_4326 = gpd.GeoSeries(query_polygon_4326, crs=4326)

query_start_time = datetime(2013, 1, 1)
query_end_time = datetime(2013, 2, 1)

Let’s take a look at where we are in Antarctica!

In the figure below:
- The red rectangle shows our area of interest
- The background is a satellite image of West Antarctica’s coastline - giving us a clear view of the icy landscape we’re studying.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

ax.plot(*query_polygon_4326.exterior.xy, c='r', lw=3)
ax.set_xlim(left=-150, right=0)
ax.set_ylim(bottom=-85, top=-55)

ax.set_ylabel('latitude  [°]', fontsize=20)
ax.set_xlabel('longitude  [°]', fontsize=20)
ax.set_aspect('equal')

ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=3, crs=4326, attribution=False)

## First query of point data: POCA points

The measurements included in the CryoTEMPO Land Ice Thematic Product are processed with the Point Of Closest Approach (POCA) algorithm. For this reason, we'll indicate the data obtained with this first query with the prefix `poca`.


Before querying the actual data, we can initially check what information about the dataset is available. For example we can look at the time span of the entire dataset and the columns it contains.
It can be very useful to know in advance what each column represents, the units and the range of values associated with each of them etc.

In [ ]:
# this is the name of the dataset we are going to use
dataset_name = 'CryoTEMPO Land Ice Thematic Product Baseline C'

available_datasets = client.list_datasets()
poca_point_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]


print(f"{dataset_name} contains data between \n"
      f"{poca_point_dataset['min_timestamp']} "
      f"and {poca_point_dataset['max_timestamp']}\n")

print(f"{dataset_name} has the following columns:")
pprint.PrettyPrinter(indent=2, width=120).pprint(poca_point_dataset['columns'])

Now lets execute the query amd preview the results! It shouldn't take more than a few seconds.

In [ ]:
poca_point_data, poca_point_data_sources = client.query_dataset(
    dataset_id=poca_point_dataset['dataset_id'],
    epsg4326_polygon=query_polygon_4326,
    min_datetime=query_start_time,
    max_datetime=query_end_time)

print(f'This query returned {len(poca_point_data)} points, drawn from {len(poca_point_data_sources)} original sources.')

poca_point_data.head()

### Understanding the POCA dataset columns

We have queried all the columns available in the database. Here’s a quick overview of what each one means:

- `elevation` is the surface elevation measurement

- `uncertainty` is the estimated uncertainty associated with the elevation measurement.

- `timestamp` indicates the time when each observation was made, in units of seconds since a standard time (normally, the 1st January 1970 at midnight).

- `geometry` contains the spatial coordinates (longitude and latitude), stored as a shapely.Point object.

- `reference_dem` indicates the surface elevation reported by a Digital Elevation Model at the same location of the measurement. Often we calculate the difference between the measured elevation and the reference DEM, hence removing the topography and allowing to compare elevation changes at different locations.

Now, let’s visualise the data!
The figure in the next cell shows each measurement as a dot located using geometry, and colored based on its elevation value 🏔️. This gives us a first glance at the spatial patterns in the dataset.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='7%', pad=0.15)
scatter_plot = ax.scatter(
    x=poca_point_data.geometry.x,
    y=poca_point_data.geometry.y,
    c=poca_point_data.elevation,
    s=2)
ax.set_xlim(left=query_polygon_4326.bounds[0], right=query_polygon_4326.bounds[2])
ax.set_ylim(bottom=query_polygon_4326.bounds[1], top=query_polygon_4326.bounds[3])
cbar = fig.colorbar(scatter_plot, cax=cax, orientation='vertical')
cbar.set_label('Elevation  [m]', rotation=-90, fontsize=16, labelpad=30)
ax.set_ylabel('latitude  [°]', fontsize=20)
ax.set_xlabel('longitude  [°]', fontsize=20)
ax.set_aspect('equal')

We can also add a little map to understand where the area of interest is.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)

ax.plot(*query_polygon_4326.exterior.xy, c='r', lw=3)
scatter_plot = ax.scatter(
    x=poca_point_data.geometry.x,
    y=poca_point_data.geometry.y,
    c=poca_point_data.elevation,
    s=2)
ax.set_xlim(left=query_polygon_4326.bounds[0] - 1.5, right=query_polygon_4326.bounds[2] + 1.5)
ax.set_ylim(bottom=query_polygon_4326.bounds[1] - 1.5, top=query_polygon_4326.bounds[3] + 1.5)

cbar = fig.colorbar(scatter_plot, cax=cax, orientation='vertical')
cbar.set_label('Elevation  [m]', rotation=-90, fontsize=20, labelpad=30)
ax.set_ylabel('latitude  [°]', fontsize=20)
ax.set_xlabel('longitude  [°]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=4326, attribution=False)

thumbnail_ax = inset_axes(ax, width='35%', height='15%', loc='upper left')
thumbnail_ax.set_ylim(-80, -60)
thumbnail_ax.set_xlim(-125, -45)
thumbnail_ax.tick_params(bottom=False, left=False, labelbottom=True, labelleft=False, labelright=True)
for spine in ['bottom', 'top', 'right', 'left']:
    thumbnail_ax.spines[spine].set_color('red')
thumbnail_ax.plot(*query_polygon_4326.exterior.xy, c='r', lw=3)
ctx.add_basemap(thumbnail_ax, source=ctx.providers.Esri.WorldImagery, zoom=4, crs=4326, attribution=False)

## Query of point data: Swath points

Let's now query the **CryoTEMPO-EOLIS Point Product**, that also contains elevation point measurements, but generated with the [swath processing method](https://earth.esa.int/eogateway/news/cryosats-swath-processing-technique).

More information can be found in the EOLIS Product Handbook at [this link](https://cryotempo-eolis.org/product-overview/).

In [ ]:
# this is the name of the dataset we are going to use
dataset_name = 'CryoTEMPO-EOLIS Point Product'

swath_point_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

---

**❓ Question 1**

---


**📝 Before You Run the Next Cell: A Quick Question!**

Before we run a query on our dataset, let’s pause for a moment and ask an important question:

- *What kind of data does this dataset actually contain?*

Knowing what variables are available - and the time period they cover - is *really* helpful. It helps you decide what kind of query you want to run and how to set it up.

To figure this out, we can check the **metadata** for the dataset. You should have already retrieved it using `.list_datasets()` in the previous cells. The metadata gives you a summary of what’s inside the dataset.

> **Question**: Look at the metadata for the **CryoTEMPO-EOLIS Point Product** (called `swath_point_dataset`).  
> - What columns (variables) does this dataset include?
> - What is the maximum timestep covered by the dataset?

Take a look at the output in the cell above to find the answers.

<details>
<summary>💡 Click here for a hint</summary>

You can access parts of the metadata like this:

- Use `swath_point_dataset['Columns']` to list the available columns (variables)
- Use `swath_point_dataset['max_timestep']` to find the maximum timestep covered by the dataset

</details>

In [ ]:
swath_point_data, swath_point_data_sources = client.query_dataset(
    dataset_id=swath_point_dataset['dataset_id'],
    epsg4326_polygon=query_polygon_4326,
    min_datetime=query_start_time,
    max_datetime=query_end_time)
print(f'This query returned {len(swath_point_data)} points, drawn from {len(swath_point_data_sources)} original sources.')

Note the remarkable difference in the number of points retrieved - 5000 POCA points vs. more than one million Swath points, over the same area and within the same time window.

Most of the columns are similar to the POCA dataset. `x` and `y` here indicate the projected spatial coordinates, in **Antarctic Polar Stereographic (EPSG: 3031)**.

In [ ]:
swath_point_data.head()

Let's plot the POCA data and the Swath data next to each other.

📌 Note: This may take up to 20 seconds due to the large volume of Swath data!

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True, figsize=(20, 8))
ax = axes[0]
poca_scatter_plot = ax.scatter(
    x=poca_point_data.geometry.x,
    y=poca_point_data.geometry.y,
    c=poca_point_data.elevation,
    s=2)
ax.set_xlim(left=query_polygon_4326.bounds[0], right=query_polygon_4326.bounds[2])
ax.set_ylim(bottom=query_polygon_4326.bounds[1], top=query_polygon_4326.bounds[3])
ax.set_xlabel('longitude  [°]', fontsize=20)
ax.set_ylabel('latitude  [°]', fontsize=20)
ax.set_aspect('equal')
ax.set_title('POCA', fontsize=30)

ax = axes[1]
swath_scatter_plot = ax.scatter(
    x=swath_point_data.geometry.x,
    y=swath_point_data.geometry.y,
    c=swath_point_data.elevation,
    s=2)
ax.set_xlim(left=query_polygon_4326.bounds[0], right=query_polygon_4326.bounds[2])
ax.set_ylim(bottom=query_polygon_4326.bounds[1], top=query_polygon_4326.bounds[3])
ax.set_xlabel('longitude  [°]', fontsize=20)
ax.set_aspect('equal')
ax.set_title('Swath', fontsize=30)

fig.subplots_adjust(right=0.89, hspace=0.05, wspace=0.08)
cax = fig.add_axes([0.92, 0.15, 0.025, 0.7])
cbar = fig.colorbar(swath_scatter_plot, cax=cax, orientation='vertical')
cbar.set_label('Elevation  [m]', rotation=-90, fontsize=16, labelpad=30)

---

**❓ Question 2**

---

**🔍 Quick Check: Comparing Swath and POCA Points**

Now that you've run the Swath and POCA queries, let’s do a quick comparison:

> **Question:** What is the ratio between the number of Swath points and the number of POCA points?

This will help you understand how much data each product type gives you, and why you might choose one over the other in different situations.

<details>
<summary>💡 Click here for a hint</summary>

Look at the print statements that appeared after the query finished running. They tell you how many points were retrieved for each product.

Try calculating the ratio:  `number of Swath points ÷ number of POCA points`

</details>

# Gridding Radar Altimetry Data

So far, we’ve seen that CryoSat-2 gives us point measurements, but those points are:

- Not evenly spaced
- Change position with each satellite pass

This irregular spacing can make the data tricky to work with directly.

To make it easier to extract information from the data, we often aggregate the points onto a grid (a process we refer to as **gridding**). Gridding helps simplify analysis and allows for consistent comparisons across time and space.

In the rest of the notebook, instead of working with longitude and latitude coordinates, we will reproject the data in **Antarctic Polar Stereographic coordinates (EPSG: 3031)**.

The function defined in the cell below contains a very simple method to do this.
- First of all, we choose a resolution for the grid. The resolution is the same in both x and y spatial coordinates, and is in units of meters for projected coordinates.
- Then we use a method, implemented in the python library pandas, to quickly join the points that belong to the same grid cell.
- Finally, we calculate the median elevation in each grid cell, considering all the points that belong to it.

In [ ]:
def grid_data(point_data_gdf: gpd.GeoDataFrame, column_name: str, grid_resolution: float) -> gpd.GeoDataFrame:
    """
    Aggregate the points on a regular grid and calculate the median value of the column of interest within each grid cell.

    Parameters
    ----------
    point_data_gdf : gpd.GeoDataFrame
        The point data.
    column_name : str
        Column name of point data to calculate the median over.
    grid_resolution : float
        Target resolution of the regular grid. The resolution is the same in both spatial coordinates.

    Returns
    -------
    gpd.GeoDataFrame
        The gridded dataset containing the median value of the column within each grid cell.
    """
    half_grid_res = grid_resolution / 2

    # assign to each point the coordinates of the grid cell it belongs to
    point_data_gdf['x_centre'] = [math.floor(x / grid_resolution) * grid_resolution + half_grid_res
                                  for x in point_data_gdf.geometry.x]
    point_data_gdf['y_centre'] = [math.floor(y / grid_resolution) * grid_resolution + half_grid_res
                                  for y in point_data_gdf.geometry.y]

    # use the pandas.DataFrame method "groupby" to aggregate the points that belong to the same grid cell
    grid_cell_df = point_data_gdf.groupby(by=['x_centre', 'y_centre'])[column_name].agg('median').reset_index()

    # construct another DataFrame where for each grid cell we retain the mean elevation
    gridded_data_gdf = gpd.GeoDataFrame({
        f'median_{column_name}': grid_cell_df[column_name],
        'geometry': box(grid_cell_df.x_centre - half_grid_res,
                        grid_cell_df.y_centre - half_grid_res,
                        grid_cell_df.x_centre + half_grid_res,
                        grid_cell_df.y_centre + half_grid_res)})

    return gridded_data_gdf

Let's use the function above to derive a gridded dataset over the area of interest!

In [ ]:
# define the EPSG code of the Antarctic Polar Stereographic Projection
aps_epsg_code = 3031
# grid the data on a regular grid with 1km resolution
grid_resolution_m = 1000
# first reproject the point data to Antarctic Polar Stereographic
swath_point_data_3031 = swath_point_data.to_crs(aps_epsg_code)
# run the gridding
gridded_data_3031 = grid_data(swath_point_data_3031.copy(), 'elevation', grid_resolution_m)

# also reproject the area of interest
query_polygon_3031 = query_extent_4326.to_crs(aps_epsg_code).iloc[0]

# plot the gridded data and the large scale map in the inset
fig, ax = plt.subplots(figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)
gridded_data_3031.plot(
    'median_elevation', ax=ax,
    cmap='viridis', cax=cax, legend=True,
    legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})
ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e5, right=query_polygon_3031.bounds[2] + 1e5)
ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e5, top=query_polygon_3031.bounds[3] + 1e5)
fig.get_axes()[1].set_ylabel('Median Elevation in Grid Cell [m]', rotation=-90, fontsize=20, labelpad=30)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_xlabel('Easting  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=aps_epsg_code, attribution=False)

thumbnail_ax = inset_axes(ax, width='30%', height='20%', loc='upper left')
thumbnail_ax.tick_params(bottom=False, left=False, labelbottom=True, labelleft=False, labelright=True)
for spine in ['bottom', 'top', 'right', 'left']:
    thumbnail_ax.spines[spine].set_color('red')
thumbnail_ax.plot(*query_polygon_3031.exterior.xy, c='r', lw=2)
thumbnail_ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e6, right=query_polygon_3031.bounds[2] + 1e6)
thumbnail_ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e6, top=query_polygon_3031.bounds[3] + 1e6)
thumbnail_ax.set_aspect('equal')
ctx.add_basemap(thumbnail_ax, source=ctx.providers.Esri.WorldImagery, zoom=4, crs=aps_epsg_code, attribution=False)

---

**❓ Question 3**

---

**📊 Try It Out: Change the Grid Resolution**

Let’s experiment a bit with how the data is gridded.

Change the value of the variable `grid_resolution_m` to `5000` (which is 5 km).

You can do this in the cell above and then re-run the cell and take a look at the output!

> **Question**: What do you observe? 👀
> - How does the gridded data look different compared to before?
> - Does changing the grid size affect the level of detail?

Take a moment to explore, and jot down anything you notice!

## POCA vs Swath: Not Just A Difference in Data Volume!
CryoSat-2's SIRAL radar works at **Ku-band** (~2.2 cm wavelength), which can penetrate into dry snow.

 - **POCA** retrieves the **strongest radar echo**, usually from the very top of the snowpack - especially when the surface is smooth and flat.

- **Swath** processing combines returns from a wider area using interferometry. This means it integrates **multiple weaker reflections**, not just the brightest one, often capturing signals from slightly deeper layers within the snowpack.

So even at the same spot, POCA and Swath can give slightly **different elevation values** because they’re sensing **different depths** in the snow!

Lets grid both the Swath and POCA data, and look at their elevation differences!

In [ ]:
# grid the data on a regular grid with 2km resolution - slightly resolution due to the lower POCA data volume
grid_resolution_m = 2000
gridded_poca_data_3031 = grid_data(poca_point_data.to_crs(aps_epsg_code), 'elevation', grid_resolution_m)
gridded_swath_data_3031 = grid_data(swath_point_data.to_crs(aps_epsg_code), 'elevation', grid_resolution_m)

# join the swath and poca data for comparison
gridded_data_joined = gridded_poca_data_3031.sjoin(gridded_swath_data_3031, how='left')

# calculate the elevation difference between the two grids
gridded_data_joined['elevation_difference'] = gridded_data_joined['median_elevation_left'] - gridded_data_joined['median_elevation_right']

# remove outliers - gridding directly on Elevation can be quite noisy, especially when considering large spatial extents such as 2km
# think about how much the terrain in Edinburgh changes within a similar area!
gridded_data_joined = gridded_data_joined[gridded_data_joined['elevation_difference'].abs() < 10]

# now plot the elevation differences
fig, ax = plt.subplots(figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)
gridded_data_joined.plot(
    'elevation_difference', ax=ax,
    vmin=-15, vmax=15, cmap='RdYlBu',
    cax=cax, legend=True, legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})
fig.get_axes()[1].set_ylabel('Elevation Difference (POCA - Swath) [m]', rotation=-90, fontsize=20, labelpad=30)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_xlabel('Easting  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=aps_epsg_code, attribution=False)

---

**❓ Question 4**

---


**📐 Calculating the Median Elevation Difference**

Now that we’ve calculated the elevation difference between POCA and Swath elevations (stored in the `elevation_difference` column of `gridded_data_joined` which is a GeoPandas dataframe), let’s take it a step further.

> **Question**:
Can you calculate the *median* elevation difference across all grid cells?

This will give you an idea of the typical offset between the two measurement types.

<details>
<summary>💡 Click here for a hint</summary>
Pandas has some handy built-in statistical functions that can help you with this.  
You can read about how to calculate the median in Pandas here:  
📚 <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.median.html" target="_blank">pandas.Series.median() documentation</a>
</details>

## Query the CryoTEMPO-EOLIS Gridded product

The CryoTEMPO-EOLIS dataset comes in two forms: one is the **Point Product**, queried and manually gridded above, and another one is the **Gridded Product**.

The CryoTEMPO-EOLIS Gridded Product is derived from the Point Product, with an algorithm which is roughly similar to the one defined in the function above.

Let's obtain one year (all 2013) of gridded data over the same area.

In [ ]:
# this is the name of the dataset we are going to use
dataset_name = 'CryoTEMPO-EOLIS Gridded Product'

swath_gridded_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

query_start_time = datetime(2013, 1, 1)
query_end_time = datetime(2014, 1, 1)

swath_gridded_data, swath_gridded_data_sources = client.query_dataset(
    dataset_id=swath_gridded_dataset['dataset_id'],
    epsg4326_polygon=query_polygon_4326,
    min_datetime=query_start_time,
    max_datetime=query_end_time)
print(f'This query returned {len(swath_gridded_data)} grid cells, drawn from {len(swath_gridded_data_sources)} original sources.')

Now let's select one month of data (**January 2013**) within that year and plot the elevation and the associated uncertainty.

In [ ]:
# update geometries to show grid cell extent
half_res = swath_gridded_data_sources[0]['source_information']['geospatial_resolution'] / 2
swath_gridded_data['geometry'] = box(
    swath_gridded_data.x_bnds_min,
    swath_gridded_data.y_bnds_min,
    swath_gridded_data.x_bnds_max,
    swath_gridded_data.y_bnds_max
)

# extract the data for January 2013
swath_gridded_data_january = swath_gridded_data.loc[swath_gridded_data.timestamp == datetime(2013, 1, 15).timestamp()]

# create a plot with two subplots, one for the elevation and one for the uncertainty on the elevation
fig, axes = plt.subplots(ncols=1, nrows=2, sharex=True, sharey=True, figsize=(15, 20))

# first subplot
ax = axes[0]
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)

# plot the elevation data
elevation_plot = swath_gridded_data_january.plot(
    'elevation', ax=ax, cmap='viridis', cax=cax, legend=True,
    legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})

# setup the colorbar
fig.get_axes()[2].set_ylabel('gridded elevation  [m]', fontsize=20, rotation=-90, labelpad=30)

# bounds, axes labels and background map
ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e5, right=query_polygon_3031.bounds[2] + 1e5)
ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e5, top=query_polygon_3031.bounds[3] + 1e5)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=aps_epsg_code, attribution=False)

# second subplot
ax = axes[1]
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)

# plot the elevation data
uncertainty_plot = swath_gridded_data_january.plot(
    'uncertainty', ax=ax, cmap='Greens', cax=cax, legend=True,
    legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})

# setup the colorbar
fig.get_axes()[3].set_ylabel('uncertainty  [m]', fontsize=20, rotation=-90, labelpad=30)

# bounds, axes labels and background map
ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e5, right=query_polygon_3031.bounds[2] + 1e5)
ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e5, top=query_polygon_3031.bounds[3] + 1e5)
ax.set_xlabel('Easting  [m]', fontsize=20)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=aps_epsg_code, attribution=False)

# bring the subplots a little closer to each other
fig.subplots_adjust(hspace=0.07)

Now extract the gridded data for <u>**December 2013**</u> and plot it.

In [ ]:
swath_gridded_data_december = swath_gridded_data.loc[swath_gridded_data.timestamp == datetime(2013, 12, 15).timestamp()]

fig, ax = plt.subplots(figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)

elevation_plot = swath_gridded_data_december.plot(
    'elevation', ax=ax, cmap='viridis', cax=cax, legend=True,
    legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})
fig.get_axes()[1].set_ylabel('gridded elevation  [m]', fontsize=20, rotation=-90, labelpad=30)
ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e5, right=query_polygon_3031.bounds[2] + 1e5)
ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e5, top=query_polygon_3031.bounds[3] + 1e5)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_xlabel('Easting  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=3031, attribution=False)

## Elevation change

It's hard to see any change between the two datasets, given the range of elevation measurements that goes from 300 m to 1300 m. Let's then calculate the difference between the two datasets, grid cell by grid cell.

In [ ]:
# first, merge the datasets such that the data associated to a given grid cell is all on the same row
# the columns with the same name in the two datasets, for example "elevation" or "uncertainty", will have a suffix appended to their name to distinguish them
swath_gridded_data_merged = pd.merge(swath_gridded_data_january, swath_gridded_data_december, on=['x', 'y'], how='inner', suffixes=['_jan', '_dec'])

# the two "geometry" columns are exactly the same, just select one of the two as the "geometry" to use in the plotting
swath_gridded_data_merged.set_geometry('geometry_jan', inplace=True)

# calculate the elevation change between December 2013 and January 2013
swath_gridded_data_merged['elevation_change'] = swath_gridded_data_merged.elevation_dec - swath_gridded_data_merged.elevation_jan

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)

elevation_change_plot = swath_gridded_data_merged.plot(
    'elevation_change', ax=ax, cmap='RdYlBu', cax=cax, legend=True, vmin=-10, vmax=10,
    legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})
fig.get_axes()[1].set_ylabel('Elevation Change  [m]', fontsize=20, rotation=-90, labelpad=30)
ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e5, right=query_polygon_3031.bounds[2] + 1e5)
ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e5, top=query_polygon_3031.bounds[3] + 1e5)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_xlabel('Easting  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=3031, attribution=False)

---

**❓ Question 5**

---


**🌍 Investigating Significant Elevation Drops**

You now have a GeoPandas dataframe called `swath_gridded_data_merged`, which contains:

- A column called `elevation_change`  
  → This shows the difference in elevation between **December 2013** and **January 2013**  
  → It was calculated by subtracting January elevations from December elevations  

> **Question**:  
How many grid cells report an elevation change **lower than -10 metres** over this time period?  
And what is the **total area** covered by these grid cells?

These might indicate areas with unusually large elevation losses, which could be important for further analysis.

<details>
<summary>💡 Click here for a hint</summary>

To answer this:

1. **Filter the GeoDataFrame** to find rows where `elevation_change < -10`  
2. Use the `.area` attribute of the geometry column to calculate each grid cell's area (this will be in metres squared due to our choice of coordinate reference system!)
3. Use `.sum()` to get the **total** area of those cells

Useful links:
- <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing" target="_blank">How to filter a DataFrame using Boolean indexing (Pandas)</a>  
- <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.area.html" target="_blank">.area attribute docs (GeoPandas)</a>

</details>

**What’s Going On Here?**

Something interesting is happening in the data!

We’re seeing significant elevation changes in a relatively short time:

- In the center of the plot, there’s a drop of more than **12 meters in less than a year** - shown by the dark red patch.

- Similar patterns (though less extreme, around 4–5 meters) appear in two other regions during 2013.

Let's dive more into where these changes are coming from!

# Subglacial lakes

Here we are observing changes that are not happening on the surface, but **beneath** the ice sheet.
At the boundary between ice and the underlying bedrock, liquid water can exist due to the high pressure. This water forms a network of rivers and lakes that experience cycles of recharge and discharge. During these cycles, the surface elevation of the ice sheet changes accordingly, for example with large localised apparent thinning during discharge events.

The areas with intense negative signal in the map above correspond to some of the subglacial lakes belonging to the network beneath the Thwaites glacier, that went through an intense discharge event during 2013, studied in scientific publications such as [Malczyk et al. (2020)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2020GL089658), [Malczyk et al. (2023)](https://www.cambridge.org/core/journals/journal-of-glaciology/article/constraints-on-subglacial-melt-fluxes-from-observations-of-active-subglacial-lake-recharge/06FB1F23816324D75FE46141C5DF4014) and [Gourmelen et al. (2025)](https://www.nature.com/articles/s41467-025-57417-1).

Let's have a closer look at these subglacial lakes.

## Download the file containing the subglacial lakes extent

Thanks to studies such as Malczyk et al. (2020), the locations of these lakes are relatively well known. The polygons corresponding to the extent of the subglacial lakes reported in Malczyk et al. (2020) are stored in a file on Google Drive. Run the next cell to download this file ("1UVFhKVDVPndr5HVoadE7lVeWZNDuAEOj" is the ID of the file).

If you click on the "Files" icon in the left bar, you will see the file in the local storage associated with this Colab session.

In [ ]:
! gdown 1UVFhKVDVPndr5HVoadE7lVeWZNDuAEOj

## Overlay the lakes extent onto the gridded elevation change map

First load the file.

The column `geometry` contains the extent of each lake, expressed in projected coordinates (EPSG:3031).

The column `lake_id` contains the names of the lakes, where the number indicates the distance of the lake from the grounding line of Thwaites Glacier.

In [ ]:
lakes_extent_file_path = os.path.join(os.getcwd(), 'malczyk2020_lakes_extent.gpkg')
lakes_extent_df = gpd.read_file(lakes_extent_file_path)
lakes_extent_df

The lakes nicely match the areas with large thinning we observed in the elevation change map we generated earlier.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='4%', pad=0.15)

elevation_change_plot = swath_gridded_data_merged.plot(
    'elevation_change', ax=ax, cmap='RdYlBu', cax=cax, legend=True, vmin=-10, vmax=10,
    legend_kwds={'orientation': 'vertical', 'shrink': .8, 'pad': .1})

for lake_index, lake_data in lakes_extent_df.iterrows():
    lake_polygon = lake_data.geometry
    ax.plot(*lake_polygon.exterior.xy, c='g')
    for ring in lake_polygon.interiors:
        ax.plot(*ring.xy, c='g')
    ax.text(x=lake_polygon.centroid.x + np.power(-1, lake_index) * 2e4,
            y=lake_polygon.centroid.y - np.power(-1, lake_index) * 2e4,
            s=lake_data.lake_id, fontsize=14, c='g', horizontalalignment='center', verticalalignment='center')

fig.get_axes()[1].set_ylabel('Elevation Change  [m]', fontsize=20, rotation=-90, labelpad=30)
ax.set_xlim(left=query_polygon_3031.bounds[0] - 1e5, right=query_polygon_3031.bounds[2] + 1e5)
ax.set_ylim(bottom=query_polygon_3031.bounds[1] - 1e5, top=query_polygon_3031.bounds[3] + 1e5)
ax.set_ylabel('Northing  [m]', fontsize=20)
ax.set_xlabel('Easting  [m]', fontsize=20)
ax.set_aspect('equal')
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, zoom=6, crs=3031, attribution=False)

## Time series of volume change for each subglacial lake

Now let's use CryoTEMPO-EOLIS Gridded Product to derive the **volume change** of each lake.

Remember, CryoSat-2 allows us to measure elevation changes at the surface. Knowing the extent of the lakes under the ice sheet, we can use the area and the elevation change to calculate the volume of water that has drained (during the discharge events) or that the lake has gained (during the recharge periods).

First of all, let's calculate the **time series** of mean surface elevation change. CryoTEMPO-EOLIS Gridded Product is a monthly product containing the ice surface elevation on a 2km regular grid. Each lake, having areas of a few hundreds km$^2$, contains multiple grid cells.

In the function defined below, for each month we calculate, grid cell by grid cell, the difference to the first available month, used as a reference elevation / first timestep. Subsequently, we take the monthly average of the elevation difference, weighting by the associated uncertainty.

In [ ]:
from datetime import timezone

def calculate_elevation_change_time_series(gridded_product_data: gpd.GeoDataFrame) -> pd.DataFrame:
    """
    Calculate a weighted average elevation change per month and associated errors.

    Weights are assigned using the CryoTEMPO-EOLIS gridded product uncertainty.

    Parameters
    ----------
    gridded_product_data : gpd.GeoDataFrame
        CryoTEMPO-EOLIS Gridded Product data.

    Returns
    -------
    pd.DataFrame
        DataFrame containing the weighted average elevation change, the associated error and the date of each timestep.
    """
    unique_timestamps = sorted(gridded_product_data.timestamp.unique())

    # elevation change is in reference to first gridded product
    reference_dem = gridded_product_data[gridded_product_data.timestamp == unique_timestamps[0]]

    # initialise empty lists for time series data
    weighted_mean_timeseries = []
    weighted_mean_errors = []
    for unique_timestamp in unique_timestamps:
        gridded_product_for_timestamp = gridded_product_data[gridded_product_data.timestamp == unique_timestamp]
        merged_gdf = reference_dem.sjoin(df=gridded_product_for_timestamp, how='inner')
        merged_gdf['elevation_difference'] = merged_gdf['elevation_right'] - merged_gdf['elevation_left']

        # combine uncertainties for elevation difference
        merged_gdf['elevation_difference_unc'] = np.sqrt(
            merged_gdf['uncertainty_right']**2 + merged_gdf['uncertainty_left']**2)

        # calculate average elevation change, weighted by measurement uncertainty
        weighted_mean = (np.sum(merged_gdf['elevation_difference'] / merged_gdf['elevation_difference_unc']**2)
                        / np.sum(1 / merged_gdf['elevation_difference_unc']**2))

        # calculate weighted average uncertainty
        error = np.sqrt(1 / np.sum(1 / merged_gdf['elevation_difference_unc']**2))

        weighted_mean_timeseries.append(weighted_mean)
        weighted_mean_errors.append(error)

    dates = [datetime.fromtimestamp(ts, tz=timezone.utc) for ts in unique_timestamps]

    return pd.DataFrame.from_dict({
        'date': dates,
        'timestamp': [d.timestamp() for d in dates],
        'mean_elevation_change': weighted_mean_timeseries,
        'uncertainty': weighted_mean_errors})

### Query of CryoTEMPO-EOLIS gridded data over each lake

In [ ]:
# info regarding the dataset
dataset_name = 'CryoTEMPO-EOLIS Gridded Product'
available_datasets = client.list_datasets()
eolis_gridded_product_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

# these dictionaries is where we'll save the data returned by the query
eolis_gridded_data_over_lakes = {}
sources_eolis_gridded_data_over_lakes = {}

# in order to query data from Specklia within an area, the corresponding polygon needs to be in EPSG 4326 (longitude-latitude)
lakes_extent_4326_df = lakes_extent_df.to_crs(epsg=4326)

# time span of the query: monthly data from January 2011 to August 2023
query_start_time = datetime(2011, 1, 1)
query_end_time = datetime(2023, 9, 1)

# run the queries (one per lake) - in total it should take 1-2 minutes
for _, lake_data in lakes_extent_4326_df.iterrows():
    eolis_gridded_data_over_lakes[lake_data.lake_id], sources_eolis_gridded_data_over_lakes[lake_data.lake_id] = client.query_dataset(
        dataset_id=eolis_gridded_product_dataset['dataset_id'],
        epsg4326_polygon=lake_data.geometry,
        min_datetime=query_start_time,
        max_datetime=query_end_time,
        columns_to_return=['timestamp', 'elevation', 'uncertainty'])

    print(f'The query of subglacial lake with ID {lake_data.lake_id} returned '
          f'{len(eolis_gridded_data_over_lakes[lake_data.lake_id])} grid cells, '
          f'drawn from {len(sources_eolis_gridded_data_over_lakes[lake_data.lake_id])} original sources.')

### Elevation change time series

Use the `calculate_elevation_change_time_series` function on the gridded data and then plot the resulting time series.

In [ ]:
elevation_change_time_series_over_lakes = {}

for _, lake_data in lakes_extent_df.iterrows():
    elevation_change_time_series_over_lakes[lake_data.lake_id] = calculate_elevation_change_time_series(
        eolis_gridded_data_over_lakes[lake_data.lake_id])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    time_series_df = elevation_change_time_series_over_lakes[lake_data.lake_id]
    ax.plot(time_series_df.date, time_series_df.mean_elevation_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(time_series_df.date,
                    time_series_df.mean_elevation_change - time_series_df.uncertainty,
                    time_series_df.mean_elevation_change + time_series_df.uncertainty,
                    alpha=0.5)

ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Elevation Change  [m]')
ax.legend(title='Subglacial lake ID')
plt.show()

The plot above is extremely interesting! The main features of the temporal evolution of these lakes are visible:
- the **discharge event** of 2013, that involved all four lakes;
- Thw_124, the largest subglacial lake of the network, rapidly **recharging** between 2015 and 2018;
- the **second discharge event**, in 2017, that involved Thw_142 and Thw_170.

### Query of CryoTEMPO-EOLIS gridded data in the surrounding area

However, another step is needed. Remember, while the lakes recharge and discharge, the ice sheet above can also thicken or thin due to the larger scale evolution (and sadly Thwaites glacier is thinning quite fast).

Hence, the elevation change above is actually given by two contributions: the **discharge-recharge cycle** of the lakes and the **thinning of the ice sheet**. Now let's try to disentangle the two processes and derive the net volume change only due to the lakes.

The strategy is the following:
1. for each lake, we consider an area that is a bit larger than the original lake extent (in this case, 10km wider in all directions)
2. query again the EOLIS gridded data, but eliminate all the grid cells that are within the lake extent
3. calculate the mean elevation change time series for the surrounding area of each lake
4. the difference between the mean elevation change calculated in the previous step and time series of the subglacial lake be isolated signal of the lake dynamics

In [ ]:
lakes_with_buffer_extent_df = lakes_extent_df.copy()
# let's use the 'buffer' method to extend the query area around each lake
lakes_with_buffer_extent_df['geometry'] = lakes_extent_df.geometry.buffer(10000)
# remember to convert everything to longitude-latitude
lakes_with_buffer_extent_4326_df = lakes_with_buffer_extent_df.to_crs(epsg=4326)

In [ ]:
# we save the grid cells over lake+buffer here
eolis_gridded_data_with_buffer = {}
sources_eolis_gridded_data_with_buffer = {}

# and the grid cells in the area surrounding each lake here
eolis_gridded_data_only_buffer = {}

for _, lake_data in lakes_with_buffer_extent_4326_df.iterrows():
    eolis_gridded_data_with_buffer[lake_data.lake_id], sources_eolis_gridded_data_with_buffer[lake_data.lake_id] = client.query_dataset(
        dataset_id=eolis_gridded_product_dataset['dataset_id'],
        epsg4326_polygon=lake_data.geometry,
        min_datetime=query_start_time,
        max_datetime=query_end_time,
        columns_to_return=['timestamp', 'elevation', 'uncertainty'])

    print(f'The query of subglacial lake + buffer with ID {lake_data.lake_id} returned '
          f'{len(eolis_gridded_data_with_buffer[lake_data.lake_id])} grid cells, '
          f'drawn from {len(sources_eolis_gridded_data_with_buffer[lake_data.lake_id])} original sources.')

    # keep all the grid cells that are not within the lake extent (i.e., that were not returned in the first query)
    eolis_gridded_data_only_buffer[lake_data.lake_id] = pd.concat(
        [eolis_gridded_data_with_buffer[lake_data.lake_id], eolis_gridded_data_over_lakes[lake_data.lake_id]]).drop_duplicates(keep=False)

    # sanity check: the number of points for the buffer zone should match the difference between the second query and the first
    assert len(eolis_gridded_data_only_buffer[lake_data.lake_id]) == (len(eolis_gridded_data_with_buffer[lake_data.lake_id]) - len(eolis_gridded_data_over_lakes[lake_data.lake_id]))
    print(f'The buffer around lake with ID {lake_data.lake_id} '
          f'contains {len(eolis_gridded_data_only_buffer[lake_data.lake_id])} grid cells, as expected.')

### Elevation change time series in the surrounding area

In [ ]:
elevation_change_time_series_over_buffer = {}

for _, lake_data in lakes_extent_df.iterrows():
    elevation_change_time_series_over_buffer[lake_data.lake_id] = calculate_elevation_change_time_series(
        eolis_gridded_data_only_buffer[lake_data.lake_id])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    time_series_df = elevation_change_time_series_over_buffer[lake_data.lake_id]
    ax.plot(time_series_df.date, time_series_df.mean_elevation_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(time_series_df.date,
                    time_series_df.mean_elevation_change - time_series_df.uncertainty,
                    time_series_df.mean_elevation_change + time_series_df.uncertainty,
                    alpha=0.5)

ax.set_xlabel('date')
ax.set_ylabel('cumulative elevation change  [m]')
ax.legend(title='Subglacial lake ID')
plt.show()

---

**❓ Question 6**

---


**🧊 Subglacial Lakes and Surface Thinning**

Let’s dig into the spatial patterns in elevation change around subglacial lakes.

> **Question**:  
In the area surrounding which **subglacial lake** has there been the most thinning?  
Do you notice any relationship between **elevation change** and the **proximity to the ice sheet margin**?

These patterns can tell us something about the underlying processes driving surface changes.

<details>
<summary>💡 Click here for a hint</summary>

Have a look at the map shown a few cells above to locate the subglacial lakes.  
The **lake IDs** are a clue: the number in the ID indicates how far each lake is from the **ice sheet margin** (lower numbers = closer to the edge).

Try comparing elevation change around lakes with lower ID numbers vs higher ones. See any trends?

</details>

<details>
<summary>📉 Click to reveal insights underlying processes driving surface change </summary>

**Why Is Surface Thinning Stronger Near the Margins of an Ice Sheet?**

---

🌊 **Proximity to Ocean & Warmer Air**

Marginal regions are more exposed to **warm ocean water** and **higher air temperatures**, which increase both basal and surface melting.
Thwaites is experiencing **rapid retreat at its grounding line**, which leads to **faster ice flow** and **surface lowering** in adjacent areas.

---

↘️ **Faster Ice Flow Near the Margins**

Ice often flows **more quickly near the edges** due to steeper slopes and reduced friction. This leads to **dynamic thinning** which outpaces accumulation from snowfall, leading to net elevation loss.

---

🧱 **Loss of Ice Shelf Support**

If an ice shelf near the margin **thins or collapses**, glaciers feeding into it can **accelerate**, triggering inland thinning through **loss of buttressing** - this is a key feature in Thwaites!

</details>

### Finally, volume change over the subglacial lakes

The plot above shows that
1. the ice sheet is thinning, hence it's important to remove this thinning signal to get the net elevation change due to the lakes
2. the ice sheet thinning is not homogeneous, so comparing the elevation change for the different lakes only based on the first time series would lead to biased results

Now, let's calculate the net elevation change due to the lakes' discharge/recharge cycles and convert it to a volume change.

In [ ]:
elevation_change_time_series = {}

for _, lake_data in lakes_extent_df.iterrows():

    elevation_change_time_series_df = pd.merge(
        elevation_change_time_series_over_lakes[lake_data.lake_id],
        elevation_change_time_series_over_buffer[lake_data.lake_id],
        how='inner', on='date', suffixes=['_lake', '_buffer'])
    # calculate the net elevation difference of the subglacial lakes and propagate the uncertainty
    elevation_change_time_series_df['net_mean_elevation_change'] = \
        elevation_change_time_series_df.mean_elevation_change_lake - elevation_change_time_series_df.mean_elevation_change_buffer
    elevation_change_time_series_df['net_uncertainty_mean_elevation_change'] = \
        np.sqrt(elevation_change_time_series_df.uncertainty_lake**2 + elevation_change_time_series_df.uncertainty_buffer**2)
    elevation_change_time_series[lake_data.lake_id] = elevation_change_time_series_df.copy()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    time_series_df = elevation_change_time_series[lake_data.lake_id]
    ax.plot(time_series_df.date, time_series_df.net_mean_elevation_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(time_series_df.date,
                    time_series_df.net_mean_elevation_change - time_series_df.net_uncertainty_mean_elevation_change,
                    time_series_df.net_mean_elevation_change + time_series_df.net_uncertainty_mean_elevation_change,
                    alpha=0.5)

ax.set_xlabel('date')
ax.set_ylabel('net elevation change  [m]')
ax.legend(title='Subglacial lake ID')
plt.show()

In order to transform surface elevation change to volume change, a fairly good first-order approximation is to multiply by the area of each subglacial lake.
For the latter, we can make use of the `area` attribute of `Polygon` objects.

📌 Note: The volume in this context is normally expressed in km$^3$. Surface elevation change is expressed in metres. The lakes extents are in Antarctic Polar Stereographic coordinates, whose units are also metres. Therefore, the area of the polygons will be in m$^2$. Hence the multiplication factor 10$^{-9}$.

In [ ]:
m3_to_km3_conversion_factor = 1e-9

volume_change_time_series = {}
for _, lake_data in lakes_extent_df.iterrows():
    elevation_change_df = elevation_change_time_series[lake_data.lake_id]
    volume_change_time_series[lake_data.lake_id] = pd.DataFrame.from_dict({
        'date': elevation_change_df.date,
        'timestamp': elevation_change_df.timestamp_lake,
        'volume_change': m3_to_km3_conversion_factor * elevation_change_df.net_mean_elevation_change * lake_data.geometry.area,
        'uncertainty_volume_change': m3_to_km3_conversion_factor * elevation_change_df.net_uncertainty_mean_elevation_change * lake_data.geometry.area})

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for _, lake_data in lakes_extent_df.iterrows():

    volume_change_df = volume_change_time_series[lake_data.lake_id]
    ax.plot(volume_change_df.date, volume_change_df.volume_change, lw=1, label=lake_data.lake_id)
    ax.fill_between(volume_change_df.date,
                    volume_change_df.volume_change - volume_change_df.uncertainty_volume_change,
                    volume_change_df.volume_change + volume_change_df.uncertainty_volume_change,
                    alpha=0.5)

ax.set_xlabel('date')
ax.set_ylabel(r'lake volume change  [km$^3$]')
ax.legend(title='Subglacial lake ID')
plt.show()

---

**❓ Question 7**

---


**Volume Loss of Subglacial Lake `Thw_124`**

The figure above clearly shows the discharge-recharge cycle of subglacial lakes -
let’s quantify the volume change of a subglacial lake during the discharge event in 2013!

> **Question**:  
How much **volume** did the subglacial lake `Thw_124` lose between **April 15th 2013** and **January 15th 2014**?

<details>
<summary>💡 Click here for a hint</summary>

To find the volume loss:

1. **Filter the dataframe** to extract the `volume_change` values for those two dates
   → Use the timestamp column to select only the rows corresponding to **April 15th 2013** and **January 15th 2014**
   ```python
   vol_april_2013 = volume_change_time_series['Thw124'][volume_change_time_series['Thw124']['timestamp'] == datetime(2013, 4, 15, tzinfo=timezone.utc).timestamp()]['volume_change'].values[0]
   ```

2. **Calculate the difference**:  
   `vol_january_2014 - vol_april_2013`  
   This gives the **net volume change** between the two dates.

📌 If the result is negative, it indicates **volume loss** over that time period.

</details>

---

**❓ Question 8**

---

**📈 Challenge: Estimating Recharge Rate of a Subglacial Lake**

Let’s dive into some data analysis!

> **Question**:  
What was the **average recharge rate** of **Thw_124** between **May 2014** and **March 2017**?  
Express your answer in **km³ per year**.

<details>
<summary>💡 Click here for hints</summary>

To estimate the recharge rate:

1. **Filter the volume change dataframe** for lake Thw_124 (`volume_change_time_series['Thw_124']`) to extract dates between May 2014 and March 2017
   ```python
    filtered_df = volume_change_time_series['Thw124'][(volume_change_time_series['Thw124']['timestamp'] >= datetime(2014, 5, 15, tzinfo=timezone.utc).timestamp()) & (volume_change_time_series['Thw124']['timestamp'] <= datetime(2017, 3, 15, tzinfo=timezone.utc).timestamp())]
    ```

2. **Fit a linear trend** to the volume change time series for Thw_124 using the **least squares method**.
   - Try using [`numpy.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) to fit a straight line: `volume = rate × time + offset`
   - The slope of that line will give you an **average rate of change** in volume.
   - Note: you should use the timestamp column for you time axis.

3. **Convert the result to km³/yr**
   - Make sure the **volume units** are in **km³**
   - Check the **time axis**: the timestamps in units of seconds - convert accordingly to get a result in **per year**

</details>

Have a look at Figure 3 in [Malczyk et al. (2023)](https://www.cambridge.org/core/journals/journal-of-glaciology/article/constraints-on-subglacial-melt-fluxes-from-observations-of-active-subglacial-lake-recharge/06FB1F23816324D75FE46141C5DF4014). It looks fairly similar, doesn't it?